# スキーム確認ツール for Navis(I1,I2,I3,I4,A26)
#### excelの全シートを読み込んでDFに格納

新しいフロートの生データ（テキスト版）からJAMSTECのデコード用DBに登録用のフィールドが存在するかを判定する

In [1]:
import os
import pandas as pd
import re
import termcolor
import Levenshtein # レーベンシュタイン距離ライブラリにある、ジャロ・ウインクラー距離を計算するのに使う
# jaro_dist = Levenshtein.jaro_winkler(srt1 , str2)

navis_excel = pd.read_excel('Navis.xlsx' , sheet_name=None) # sheet_name=Noneで全てのシート読み込む

## ジャロ・ウィンクラー距離法の関数

２つの文字列の類似度を返す。１で完全一致

#### Winkler, W. E. (1990). "String Comparator Metrics and Enhanced Decision Rules in the Fellegi-Sunter Model of Record Linkage". Proceedings of the Section on Survey Research Methods. American Statistical Association: 354–359.

In [2]:
def jaro_dist(str1,str2):
    return Levenshtein.jaro_winkler(str1,str2)

#### DFに格納、検索しやすいようにカラムにも名前をつける
シート名が日本語でも指定すればちゃんと読める。
必要な列を抜き出してscheme,techという新しいDFを作る

In [3]:
#print(navis_excel['技術情報'].columns)
# df.loc[行名：列名]
# df.iloc[行番号：列番号]

msgcol = navis_excel['scheme'].iloc[:,1]
i12msg = navis_excel['scheme'].iloc[:,7] # ７列目がI1/I2のmsg , 8列目はlog。以降同様にカウントする 
i3msg = navis_excel['scheme'].iloc[:,9]
i4msg = navis_excel['scheme'].iloc[:,12]

scheme = pd.concat([msgcol,i12msg,i3msg,i4msg], axis=1).rename(columns={'Unnamed: 1':'field_name' , 'Unnamed: 7':'i1i2a26' , 'Unnamed: 9':'i3X1A27' , 'Unnamed: 12':'i4'})
#print(len(scheme))
#print(scheme.iloc[7,:])
#print(scheme['i1i2a26'])

techcol = navis_excel['tech'].iloc[:,1]
i1msg = navis_excel['tech'].iloc[:,7]
i1log = navis_excel['tech'].iloc[:,8]
i2msg = navis_excel['tech'].iloc[:,9]
i2log = navis_excel['tech'].iloc[:,10]
i3msg = navis_excel['tech'].iloc[:,11]
i3log = navis_excel['tech'].iloc[:,12]
i3isus = navis_excel['tech'].iloc[:,13]
i4msg = navis_excel['tech'].iloc[:,14]
i4log = navis_excel['tech'].iloc[:,15]

tech = pd.concat([techcol,i1msg,i1log,i2msg,i2log,i3msg,i3log,i3isus,i4msg,i4log], axis=1).rename(columns={'Unnamed: 1':'field_name', 'Unnamed: 7':'i1msg' , 'Unnamed: 8':'i1log' , 'Unnamed: 9':'i2msg' , 'Unnamed: 10':'i2log' , 'Unnamed: 11':'i3msg' , 'Unnamed: 12':'i3log' , 'Unnamed: 13':'i3isus' , 'Unnamed: 14':'i4msg' , 'Unnamed: 15':'i4log'})

#print(tech.iloc[56,:])
#print(type(tech))

51


## 生データファイル読み込み
バイナリからアスキーへの変換はメーカー提供のツールによる

In [4]:
with open('i0394/0394.001.msg','r') as msg:
    msgline = msg.readlines() # prof,techで使う
    
with open('i0394/0394.001.log','r') as log:
    logline = log.readlines()

In [5]:
# msgline,loglineとも1行毎に読み込んでる。
# 安直にループさせて新しいDataFrameを作ることは出来ないので辞書リストに一度保存する（Pythonの仕様）
msgdata = {}
i = 0
for orig_msg in msgline:
    regex = r'^\$' # 先頭が＄ならその行をパラメータと判断する
    regex2 = r'^[A-Z]' # 先頭が大文字アルファベットの場合もパラメータと判断
    pattern = re.compile(regex)
    pattern2 = re.compile(regex2)

    # 行頭$スペースの2文字を除いて（の手前までをパラメータとして保存
    if (pattern.match(orig_msg) ): # ’(’以降は削除
        pos = orig_msg.find('(')
        # print(orig_msg[2:pos])
        msgdata[i] =  (orig_msg[2:pos])
        i += 1
    elif (pattern2.match(orig_msg)): # 先頭大文字、＝までもパラメータとする
        pos = orig_msg.find('=')
        #print(orig_msg[:pos])
        msgdata[i] =  orig_msg[:pos]
        i += 1

# dict_valuesをPythonの listに変換する必要はなかった。
#print(data.values())
msgdf = pd.DataFrame(msgdata.values() , columns={'field_name'})
#print(len(msgdata))

75


## logfileも同様にDataFrameにする
### 上記の様に安直にループさせて新規のDFは仕様上作れないので一度辞書リストに保存する

In [6]:
logdata = {}
i  = 0

for orig_log in logline:
# sec)が見えたらその次の文字列をパラメータとみなす
    pos = orig_log.find('sec) ')
    pos2 = orig_log.find('()')
    #print(orig_log[pos+5:pos2].split(' ')[0])
    logdata[i] = orig_log[pos+5:pos2].split(' ')[0]
    i += 1

logdf = pd.DataFrame(logdata.values(),columns={'field_name'})
#print(logdf)
#(len(scheme.dropna()))

30


### 比較する
#### schee,techのDataframeと生データを読み込んだリストで文字列比較
- scheme エクセルのスキーム情報タブ
     - カラム名　field_name　,　i1i2A26　, i3　,　i4
- tech エクセルの技術情報タブ
    - カラム名　field_name , i1msg , i1log , i2msg , i2log , i3msg , i3log , i3isus , i4msg , i4log

#### とりあえずDataframeも準備したけど辞書リストと比較しても良い
- msgdf テキスト生データのメッセージファイルに書いてあるパラメータ名のdataframe
    - カラム名 field_name
- msgdata 生データのパラメータを抜き出した辞書リストdict
- logdf テキスト生データのログファイルに書いてあるパラメータ名のdataframe
    - カラム名 field_name
- logdata 生データのパラメータを抜き出した辞書リストdict

#### dataframe は検索結果はQuery（Series）で返る
文字列が一致したIndexにはTrue、それ以外にはFalseが入ったqueryが返るので

Trueがある場合そのIndexを取得して有無を判定する（無いばあいはNoneが返る）

#### msgとスキーマ情報の比較

dataframe(scheme)の1列はSeriesになる。

queryが全部Falseだった場合はDBフィールドが無いということ。

もう一度ループを回して生データの1行と比較する（12行目あたり）

scoreを一旦保存して一番高いものを一つ表示する(result というDataFrameに保存）

In [7]:
result = pd.DataFrame(columns=['index','msg','xls','score'])

for line in msgdata.values():
    #print(line)
    query = scheme['i4'].dropna().str.startswith(line , na=False)
    #print(query.values)
    #print(scheme['i1i2a26'].dropna()) # NaNのところは削除
    if (query[query == True].first_valid_index()):
        exist = line + ' field exists.'
        print(termcolor.colored(exist,'blue'))
    else:
        nf = line + ' is not found.'
        print(termcolor.colored(nf,'red'))
        
        for index,item in scheme.dropna().iterrows():
            score = jaro_dist(str(item['i4']) , line) # 引数を入れ替えると結果が多少変わる。
            #print(line + ' is probably ' + str(item['i4']) + ' ( ' + str(round(score,2)*100) + '%)' )
            record = pd.Series([index , line , item['i4'] , score] , index=result.columns)
            result = result.append(record , ignore_index=True)
            
#print(result)


Mission configuration for Npf is not found.
AscentTimeOut field exists.
AtDialCmd field exists.
AltDialCmd field exists.
BuoyancyNudge field exists.
BuoyancyNudgeInitial field exists.
ConnectTimeOut field exists.
CpActivationP field exists.
DeepProfileDescentTime field exists.
DeepProfileBuoyancyPos field exists.
DeepProfilePressure field exists.
DownTime field exists.
FloatId is not found.
FullExtension field exists.
FullRetraction field exists.
HpvEmfK is not found.
HpvRes is not found.
MaxAirBladder field exists.
MaxLogKb field exists.
MissionPrelude field exists.
OkVacuum field exists.
PActivationBuoyancyPosition is not found.
ParkDescentTime field exists.
ParkBuoyancyPos field exists.
ParkPressure field exists.
PnPCycleLen field exists.
TelemetryRetry field exists.
TimeOfDay field exists.
UpTime field exists.
Verbosity field exists.
DebugBits field exists.
 is not found.
ParkPts:    Sep 09 2014 19:22:50  1410290570   16238  863.74  4.5244 34.1137 is not found.
Profile 0394.001 ter

In [8]:
items = len(scheme.dropna()) # Index数、この数分ジャロ・ウインクラー距離を計算したら次のマッチしなかった語句になる
kazu = int( len(result) / items ) # クエリーのリストになくnot foundで表示した数

for count in range(kazu):
    res = result[items*count : items*(count+1)]
    ranking = res.sort_values('score',ascending=False)[:3] # Score降順にソートして上から3つを表示

    if((re.match('^.',ranking.iat[0,1])) is not None  ):
        # print(ranking.iat[0,3])
        # score が０と１の時は抜く（アルゴリズム？で１００％が結構出てる。
        disp_res = str(ranking.iat[0,1]) + ' is probably ' + str(ranking.iat[0,2]) + ' ( ' + str(round(ranking.iat[0,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(ranking.iat[1,2]) + ' ( ' + str(round(ranking.iat[1,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(ranking.iat[2,2]) + ' ( ' + str(round(ranking.iat[2,3] , 2) * 100 ) + '% )' + '\n' 
    
        print( disp_res )


Mission configuration for Npf is probably MissionPrelude ( 88.0% )
 or PActivationPistonPosition ( 54.0% )
 or MaxAirBladder ( 50.0% )

FloatId is probably FullExtension ( 60.0% )
 or FullRetraction ( 59.0% )
 or AtDialCmd ( 59.0% )

HpvEmfK is probably UpTime ( 54.0% )
 or TimeOfDay ( 50.0% )
 or CpActivationP ( 48.0% )

HpvRes is probably UpTime ( 56.00000000000001% )
 or Verbosity ( 52.0% )
 or ParkPressure ( 50.0% )

PActivationBuoyancyPosition is probably PActivationPistonPosition ( 100.0% )
 or CpActivationP ( 75.0% )
 or ParkBuoyancyPos ( 68.0% )

ParkPts:    Sep 09 2014 19:22:50  1410290570   16238  863.74  4.5244 34.1137 is probably ParkPressure ( 78.0% )
 or ParkBuoyancyPos ( 70.0% )
 or ParkDescentTime ( 68.0% )

Profile 0394.001 terminated: Wed Sep 10 05:16:36 2014 is probably PnPCycleLen ( 50.0% )
 or ParkPressure ( 49.0% )
 or DeepProfileDescentTime ( 48.0% )

Discrete samples: 1 is probably DeepProfilePressure ( 60.0% )
 or DeepProfileBuoyancyPos ( 56.99999999999999% )
 

#### ジャロ・ウィンクラー距離法で文字列比較

##### 80%以上なら表示、以下のループだとフィールドがあるやつも表示してしまう。

ジャロウィンクラー距離法でNaNは文字列でないためそのまま使えない。面倒なのでdropna()でNaNは削除している。

scoreはround()で偶数丸め（四捨五入ではない）をして％表記

## debug用ブロック
for line in msgdata.values():
    for index, item in scheme.dropna().iterrows():
        score = jaro_dist(line,str(item['i4']))
        if ( score  > 0.1 and score < 1.0 ): # 80%以上なら候補を表示
            print (line + '　is probably ' + str(item['i4']) + ' ( ' + str(round(score,2)*100) + '% ) ')
        #print(line,item['i1i2a26'],index)

#print(jaro_dist('AirPressure', 4))

In [9]:
# debug用ブロック

str1 = 'AirPressure'
str2 = 'AirPres'
lev_dist = Levenshtein.distance(str1,str2)
# 標準化、長い方の文字列の長さで割る
devider = len(str1) if len(str1) > len(str2) else len(str2)
lev_dist = lev_dist / devider
# 指標を合わせる（０：完全不一致＞１：完全一致）
lev_dist = 1 - lev_dist
#print(lev_dist)

0.6363636363636364


#### 　msgと技術情報タブの比較

In [10]:
# ジャロウィンクラー距離のスコアを一度DFに保存してソートする
diff_result = pd.DataFrame(columns=['index','msg','xls','score'])

for line in logdata.values():
    #print(line)
    query = tech['i4msg'].dropna().str.startswith(line , na=False)
    #print(query.values)
    if (query[query == True].first_valid_index()):
        exist = line + ' field exists.'
        print(termcolor.colored(exist,'blue'))
    else:
        nf = line + ' is not found.'
        print(termcolor.colored(nf,'red'))

        for index,item in tech.iterrows():
            score = jaro_dist(str(item['i4msg']) , line) # 引数を入れ替えると結果が多少変わる。
            #print(score)
            #print(line + ' is probably ' + str(item['i4msg']) + ' ( ' + str(round(score,2)*100) + '%)' )
            diff_record = pd.Series([index , line , item['i4msg'] , score] , index=diff_result.columns)
            diff_result = diff_result.append(diff_record , ignore_index=True)


TelemetryInit is not found.
AirSystem is not found.
AirSystem is not found.
AirSystem is not found.
GpsServices is not found.
GpsServices is not found.
gga is not found.
GpsServices is not found.
GpsServices is not found.
GpsServices is not found.
gga is not found.
gga is not found.
GpsServices is not found.
GpsServices is not found.
CLogin is not found.
CLogin is not found.
login is not found.
CLogin is not found.
RxConfig is not found.
Rx is not found.
Rx is not found.
Rx is not found.
RxConfig is not found.
WriteVitals is not found.
UpLoadFile is not found.
Tx is not found.
TxPacket is not found.
TxPacket is not found.
Tx is not found.
UpLoadFile is not found.
UpLoadFile is not found.
Tx is not found.
Tx is not found.
UpLoadFile is not found.
UpLoad is not found.
UpLoad is not found.
logout is not found.
Telemetry is not found.
DescentInit is not found.
DescentInit is not found.
BuoyancyAdjustAbsWTO is not found.
Descent is not found.
Descent is not found.
Descent is not found.
Desc

In [11]:
# スコアの結果DF（上ではdiff_resultをソートして上から3つを表示

techlength = len(tech) # Index数、この数分ジャロ・ウインクラー距離を計算したら次のマッチしなかった語句になる
num = int( len(diff_result) / techlength ) # クエリーのリストになくnot foundで表示した数

for count in range(num):
    diff_res = diff_result[techlength*count : techlength*(count+1)]
    diff_ranking = diff_res.sort_values('score',ascending=False)[:3] # Score降順にソートして上から3つを表示
    
    #print(diff_result)
    if((re.match('^.',diff_ranking.iat[0,1])) is not None  ):
        #print(diff_ranking.iat[0,3])
        # score が０と１の時は抜く（アルゴリズム？で１００％が結構出てる。
        disp_res2 = str(diff_ranking.iat[0,1]) + ' is probably ' + str(diff_ranking.iat[0,2]) + ' ( ' + str(round(diff_ranking.iat[0,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(diff_ranking.iat[1,2]) + ' ( ' + str(round(diff_ranking.iat[1,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(diff_ranking.iat[2,2]) + ' ( ' + str(round(diff_ranking.iat[2,3] , 2) * 100 ) + '% )' + '\n' 
    
        print( disp_res2 )





TelemetryInit is probably TimeStartPark ( 63.0% )
 or TimeStopProfile ( 61.0% )
 or TimeStartProfile ( 61.0% )

AirSystem is probably AirPumpAmps ( 72.0% )
 or AirPumpVolts ( 71.0% )
 or AirBladderPressure ( 69.0% )

AirSystem is probably AirPumpAmps ( 72.0% )
 or AirPumpVolts ( 71.0% )
 or AirBladderPressure ( 69.0% )

AirSystem is probably AirPumpAmps ( 72.0% )
 or AirPumpVolts ( 71.0% )
 or AirBladderPressure ( 69.0% )

GpsServices is probably GpsFixTime ( 71.0% )
 or QuiescentAmps ( 61.0% )
 or QuiescentVolts ( 60.0% )

GpsServices is probably GpsFixTime ( 71.0% )
 or QuiescentAmps ( 61.0% )
 or QuiescentVolts ( 60.0% )

gga is probably nan ( 56.00000000000001% )
 or nan ( 56.00000000000001% )
 or nan ( 56.00000000000001% )

GpsServices is probably GpsFixTime ( 71.0% )
 or QuiescentAmps ( 61.0% )
 or QuiescentVolts ( 60.0% )

GpsServices is probably GpsFixTime ( 71.0% )
 or QuiescentAmps ( 61.0% )
 or QuiescentVolts ( 60.0% )

GpsServices is probably GpsFixTime ( 71.0% )
 or Quiesc

##### logと技術情報の比較

In [12]:
# 保存用のDFを準備
# ジャロウィンクラー距離のスコアを一度DFに保存してソートする
# Pythonは参照渡しなのでメモリ上にデータは全部残っているから変数名はキチン特別しておくこと
tech_result = pd.DataFrame(columns=['index','msg','xls','score'])


for line in logdata.values():
    #print(line)
    query = tech['i4log'].str.startswith(line , na=False)
    #print(query.values)
    if (query[query == True].first_valid_index()):
        exist = line + ' field exists.'
        print(termcolor.colored(exist,'blue'))
    else:
        nf = line + ' is not found.'
        print(termcolor.colored(nf,'red'))
        
        for index,item in tech.iterrows():
            score = jaro_dist(str(item['i4log']) , line) # 引数を入れ替えると結果が多少変わる。
            #print(score)
            #print(line + ' is probably ' + str(item['i4msg']) + ' ( ' + str(round(score,2)*100) + '%)' )
            tech_record = pd.Series([index , line , item['i4msg'] , score] , index=tech_result.columns)
            tech_result = tech_result.append(tech_record , ignore_index=True)


TelemetryInit is not found.
AirSystem is not found.
AirSystem is not found.
AirSystem is not found.
GpsServices is not found.
GpsServices is not found.
gga is not found.
GpsServices is not found.
GpsServices is not found.
GpsServices is not found.
gga is not found.
gga is not found.
GpsServices is not found.
GpsServices is not found.
CLogin is not found.
CLogin is not found.
login is not found.
CLogin is not found.
RxConfig is not found.
Rx is not found.
Rx is not found.
Rx is not found.
RxConfig is not found.
WriteVitals is not found.
UpLoadFile is not found.
Tx is not found.
TxPacket is not found.
TxPacket is not found.
Tx is not found.
UpLoadFile is not found.
UpLoadFile is not found.
Tx is not found.
Tx is not found.
UpLoadFile is not found.
UpLoad is not found.
UpLoad is not found.
logout is not found.
Telemetry is not found.
DescentInit is not found.
DescentInit is not found.
BuoyancyAdjustAbsWTO is not found.
Descent is not found.
Descent is not found.
Descent is not found.
Desc

### 類似度の高い方から３つを表示する

In [13]:
# スコアの結果DF（上ではtech_resultをソートして上から3つを表示

techloglen = len(tech) # Index数、この数分ジャロ・ウインクラー距離を計算したら次のマッチしなかった語句になる
num = int( len(tech_result) / techloglen ) # クエリーのリストになくnot foundで表示した数

for count in range(num):
    tech_res = tech_result[techloglen*count : techloglen*(count+1)]
    tech_ranking = tech_res.sort_values('score',ascending=False)[:3] # Score降順にソートして上から3つを表示
    
    #print(diff_result)
    if((re.match('^.',tech_ranking.iat[0,1])) is not None  ):
        #print(diff_ranking.iat[0,3])
        # score が０と１の時は抜く（アルゴリズム？で１００％が結構出てる。
        disp_res3 = str(tech_ranking.iat[0,1]) + ' is probably ' + str(tech_ranking.iat[0,2]) + ' ( ' + str(round(tech_ranking.iat[0,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(tech_ranking.iat[1,2]) + ' ( ' + str(round(tech_ranking.iat[1,3] , 2) * 100 ) + '% )' + '\n' \
        + ' or ' + str(tech_ranking.iat[2,2]) + ' ( ' + str(round(tech_ranking.iat[2,3] , 2) * 100 ) + '% )' + '\n' 
    
        print( disp_res3 )

TelemetryInit is probably nan ( 53.0% )
 or nan ( 53.0% )
 or msg ( 47.0% )

AirSystem is probably nan ( 42.0% )
 or nan ( 40.0% )
 or I4 ( 0.0% )

AirSystem is probably nan ( 42.0% )
 or nan ( 40.0% )
 or I4 ( 0.0% )

AirSystem is probably nan ( 42.0% )
 or nan ( 40.0% )
 or I4 ( 0.0% )

GpsServices is probably nan ( 51.0% )
 or nan ( 44.0% )
 or I4 ( 0.0% )

GpsServices is probably nan ( 51.0% )
 or nan ( 44.0% )
 or I4 ( 0.0% )

gga is probably I4 ( 56.00000000000001% )
 or nan ( 56.00000000000001% )
 or TimeStartDescent ( 56.00000000000001% )

GpsServices is probably nan ( 51.0% )
 or nan ( 44.0% )
 or I4 ( 0.0% )

GpsServices is probably nan ( 51.0% )
 or nan ( 44.0% )
 or I4 ( 0.0% )

GpsServices is probably nan ( 51.0% )
 or nan ( 44.0% )
 or I4 ( 0.0% )

gga is probably I4 ( 56.00000000000001% )
 or nan ( 56.00000000000001% )
 or TimeStartDescent ( 56.00000000000001% )

gga is probably I4 ( 56.00000000000001% )
 or nan ( 56.00000000000001% )
 or TimeStartDescent ( 56.0000000000